In [1]:
import tika
import tika.parser
from tika import parser
import requests
import urllib.parse
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#run "python -mhttp.server 8888" in the directory where your photos are

#run ifconfig in terminal to find your ip address listed for en0

images_url_usc  = 'http://10.25.179.208:8888'

images_url_usc2 = 'http://10.25.77.81:8888'

images_url_home = 'http://192.168.1.13:8888'

images_url_MOC = 'http://10.25.188.76:8888'

# Webscraping all 80,581 URLs

In [3]:
import requests
from bs4 import BeautifulSoup
import csv

webpage_url = images_url_usc2

response = requests.get(webpage_url)
html_content = response.content

soup = BeautifulSoup(html_content, 'html.parser')

image_urls_set = set()
count = 0
for link in soup.find_all('a'):
    href = link.get('href')
    if href is not None and href.startswith('http'):
        image_urls_set.add(href)
        count += 1
    elif href is not None:
        image_urls_set.add(webpage_url + '/' + href)
        count += 1
    print(f"Processed {count} links", end='\r')

image_urls = list(image_urls_set)

# Save the image URLs to a text file
with open('image_urls.txt', mode='w') as file:
    for url in image_urls:
        file.write(url + '\n')
        
print("All links processed!") # added for confirmation


All links processed!s


In [5]:
#verifying there were no mistakes durnig link extraction

with open('image_urls.txt', 'r') as f:
    num_links = len(f.readlines())

print(f'The file contains {num_links} total links.')


with open('image_urls.txt', 'r') as f:
    unique_links = set(line.strip() for line in f if line.strip())

num_unique_links = len(unique_links)

print(f'The file contains {num_unique_links} unique links.')





The file contains 80581 total links.
The file contains 80581 unique links.


# Image Captions DF

- High level strategy: create a list of key value pairs, and save it to a JSON file periodically so that if the program crashes I don't lose my progress


In [5]:
#docker run -it -p 8764:8764 uscdatascience/im2txt-rest-tika
#test_url = 'http://localhost:8764/inception/v3/caption/image?url=http://10.25.77.81:8888/Pixstory-image-164318304274781.png'

captions_results = []

# Loop through the image URLs and classify each image
for url in image_urls:

    # Construct the API URL for the image classification
    api_captions_url = 'http://localhost:8764/inception/v3/caption/image?url=' + url

    # Send the API request
    response = requests.get(api_captions_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the first listed sentence
    first_sentence = json_response['captions'][0]['sentence']

    # Add the results to the list
    captions_results.append({

        'url': url, 
        'caption':first_sentence
    })

# Create a Pandas dataframe from the list of dictionaries
captions_df = pd.DataFrame(captions_results)

# Set display options to show all columns and avoid truncation of string data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Display the dataframe
captions_df

,url,caption
0,http://192.168.1.13:8888/Pixstory-image-161668402049563.jpg,a bunch of bananas that are on a table .
1,http://192.168.1.13:8888/Pixstory-image-161681318757326.jpg,a man sitting on a couch with a laptop .
2,http://192.168.1.13:8888/Pixstory-image-161984246212221.jpeg,a man in a suit and tie holding a cell phone .
3,http://192.168.1.13:8888/Pixstory-image-162014760674045.jpeg,a bunch of birds that are flying in the air .
4,http://192.168.1.13:8888/Pixstory-image-162051250471634.jpeg,a little girl sitting in a chair holding a teddy bear .
5,http://192.168.1.13:8888/Pixstory-image-162153257364412.png,a bike parked on the side of a road .
6,http://192.168.1.13:8888/Pixstory-image-162157341332029.jpeg,a man in a suit and tie standing in a room .
7,http://192.168.1.13:8888/Pixstory-image-162194441755927.jpeg,a red fire hydrant sitting in the middle of a forest .
8,http://192.168.1.13:8888/Pixstory-image-162244448366044.jpeg,a bunch of flowers that are on a table .
9,http://192.168.1.13:8888/Pixstory-image-16227252974324.png,a close up of a person holding a pair of scissors


# testing time to get 300 image captions without threading

In [6]:
import json

# Load image URLs from file
with open('image_urls.txt') as f:
    image_urls = f.read().splitlines()

captions_results = {}

# Loop through the image URLs and classify each image
for url in image_urls:

    # Construct the API URL for the image classification
    api_captions_url = 'http://localhost:8764/inception/v3/caption/image?url=' + url

    # Send the API request
    response = requests.get(api_captions_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the first listed sentence
    first_sentence = json_response['captions'][0]['sentence']

    # Add the results to the dictionary
    captions_results[url] = first_sentence
    
    # Stop the program once there are 300 key value pairs in the dictionary
    if len(captions_results) == 300:
        break

# Write the results to a JSON file
with open('captions.json', 'w') as f:
    json.dump(captions_results, f)


#in 10 minutes in only created 24 captions --> 25 seconds per caption

KeyboardInterrupt: 

In [7]:

# Get the number of elements in the dictionary
num_elements = len(captions_results)

# Print the result
print(num_elements)


24


# testing time to get 300 image captions with multithreading 

In [8]:
import json
import threading

# Load image URLs from file
with open('image_urls.txt') as f:
    image_urls = f.read().splitlines()

captions_results = {}

# Define a function to classify the image and store the caption in the dictionary
def classify_image(url):
    # Construct the API URL for the image classification
    api_captions_url = 'http://localhost:8764/inception/v3/caption/image?url=' + url

    # Send the API request
    response = requests.get(api_captions_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the first listed sentence
    first_sentence = json_response['captions'][0]['sentence']

    # Add the results to the dictionary
    captions_results[url] = first_sentence

# Define the maximum number of threads to use
max_threads = 10

# Create a list of threads to run in parallel
threads = []
for url in image_urls:
    # Stop creating threads once there are 300
    if len(captions_results) == 300:
        break
        
    # Create a new thread to classify the image
    thread = threading.Thread(target=classify_image, args=(url,))
    
    # Start the thread
    thread.start()
    
    # Add the thread to the list of threads
    threads.append(thread)
    
    # Wait for the threads to finish if the maximum number of threads has been reached
    if len(threads) >= max_threads:
        for thread in threads:
            thread.join()
        threads = []

# Wait for any remaining threads to finish
for thread in threads:
    thread.join()

# Write the results to a JSON file
with open('captions2.json', 'w') as f:
    json.dump(captions_results, f)

#in 8 minutes it created 130 captions --> 3.7 seconds per caption


KeyboardInterrupt: 

# threads = 20

In [ ]:
import json
import threading

# Load image URLs from file
with open('image_urls.txt') as f:
    image_urls = f.read().splitlines()

captions_results = {}

# Define a function to classify the image and store the caption in the dictionary
def classify_image(url):
    # Construct the API URL for the image classification
    api_captions_url = 'http://localhost:8764/inception/v3/caption/image?url=' + url

    # Send the API request
    response = requests.get(api_captions_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the first listed sentence
    first_sentence = json_response['captions'][0]['sentence']

    # Add the results to the dictionary
    captions_results[url] = first_sentence

# Define the maximum number of threads to use
max_threads = 10

# Create a list of threads to run in parallel
threads = []
for url in image_urls:
    # Stop creating threads once there are 300
    if len(captions_results) == 300:
        break
        
    # Create a new thread to classify the image
    thread = threading.Thread(target=classify_image, args=(url,))
    
    # Start the thread
    thread.start()
    
    # Add the thread to the list of threads
    threads.append(thread)
    
    # Wait for the threads to finish if the maximum number of threads has been reached
    if len(threads) >= max_threads:
        for thread in threads:
            thread.join()
        threads = []

# Wait for any remaining threads to finish
for thread in threads:
    thread.join()

# Write the results to a JSON file
with open('captions2.json', 'w') as f:
    json.dump(captions_results, f)

#in 8 minutes it created 130 captions --> 3.7 seconds per caption

In [9]:
# Get the number of elements in the dictionary
num_elements = len(captions_results)

# Print the result
print(num_elements)

130
